#Import e autenticação

In [36]:
import tweepy as tw
# Fazer comunicação
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
# Fazer autenticação
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
import json
import pandas as pd



In [3]:
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""
cliente_id = ""
cliente_secret = ""

In [4]:
# Faz a autenticação na API

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [6]:
# Pra ver se estamos conectados
api = tw.API(auth)

In [ ]:
tweet = api.update_status("Teste API Twitter #Alura")

In [ ]:
# Informações sobre o json
tweet._json

# Pesquisando Tweets

### Histórico
* Olha o passado

In [10]:

# Iterar sobre tweets que foram postados
tweets = tw.Cursor(
    # Método pra fazer pesquisa
    api.search_tweets,
    # o que quero pesquisar
    q="machine learning",
    #desde quando
    since = "2020-01-01",
    # qual idioma
    lang='pt'
    ).items(10)

In [ ]:
for t in tweets:
  print(t.text)

In [12]:

# Iterar sobre tweets que foram postados
tweets = tw.Cursor(
    # Método pra fazer pesquisa
    api.search_tweets,
    # o que quero pesquisar
    q="python",
    #desde quando
    since = "2020-01-01",
    # qual idioma
    lang='pt'
    ).items(20)

In [ ]:
for t in tweets:
  print(t.user.screen_name)
  print(t.user.location)
  print('-----\n')

#### Pesquisando famosos

In [14]:
famosos = ['cauareymond','aguiarthur','ivetesangalo','ClaudiaLeitte','neymarjr','BruMarquezine','mariruybarbosa',
'FePaesLeme','Tatawerneck','FlaviaAleReal','julianapaes','dedesecco','SabrinaSato','ahickmann','gusttavo_lima','Anitta',
'CarolCastroReal','gio_antonelli','maisa','cleooficial','gewbank','taisdeverdade','otaviano','bernardipaloma',
'IngridGuimaraes','olazaroramos','GalisteuOficial','debranascimento','FioMattheis','moalfradique','Nandacostareal']

In [15]:
len(famosos)

31

In [ ]:
for famoso in famosos:
  tweets = tw.Cursor(api.search_users,
         q=famoso,
         since='2023-01-01').items(20)
  print('Autor: ', famoso)
  print('Imagens postadas:')

  for tweet in tweets:
    # Se tem imagem...
    if 'media' in tweet.entities:
      # Qual a url
      print(tweet.entities['media'][0]['media_url'])

  print('-----')
  print('\n')

### Streaming
* Nova forma de pesquisar: a partir de agora (ao vivo!)

* Não é mais pesquisa do passado

In [19]:
# Herança da classe da biblioteca: "tw.StreamListener"
class MinhaStreamListener(tw.StreamingClient):

  # Alterando o método
  def on_status(self, status):
    print(status.user.screen_name)
    print(status.text)
    print('-----')
    print('\n')

In [ ]:
minhaStream = tw.StreamResponse(auth = auth,
                        # Qual classe que vai trabalhar conforme eventos (capturas)
                        listener=MinhaStreamListener())

In [ ]:
# Filtra infos relacionadas aos famosos
minhaStream.filter(track=famosos)

# Autenticando na Azure

In [26]:
!pip install azure-cognitiveservices-vision-computervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [32]:
chave_1 = "bab8794cb7664cb5bed597b6544b96f8"
credenciais = CognitiveServicesCredentials(chave_1)

In [33]:
endpoint = "https://alura-mariana.cognitiveservices.azure.com/"
client = ComputerVisionClient(endpoint, credenciais)

In [34]:
client.api_version

'3.2'

![](http://pbs.twimg.com/media/ECx6hK-WwAAPzeE.jpg)

In [ ]:
url = "http://pbs.twimg.com/media/ECx6hK-WwAAPzeE.jpg"

analise_de_imagem = client.analyze_image(url,
                                         # palavras que definem a imagem
                                         VisualFeatureTypes.tags)

In [ ]:
for tag in analise_de_imagem.tags:
  print(tag)

A Azure disponibiliza um [tutorial](https://learn.microsoft.com/pt-br/azure/ai-services/Computer-vision/how-to/analyze-video) para fazer análise de vídeos quase em tempo real.

### Buscando Celebridades

In [ ]:
# domain = Analisar coisas famosas
analise_celebridades = client.analyze_image_by_domain(
                                                      # o que famoso
                                                      "celebrities",
                                                      url,
                                                      "en")


In [ ]:
for celebridade in analise_celebridades.result["celebrities"]:
  print(celebridade['name'])
  print(celebridade['confidence'])

In [ ]:
analise_celebridades.result["celebrities"]

In [ ]:
descricao = client.describe_image(url,
                                  # numero maximo de descrições
                                  3,
                                  "en")

In [ ]:
# Descrição da imagem
for caption in descricao.captions:
  print(caption.text)
  print(caption.confidence)

In [ ]:
descricao.captions[0].text

# Streaming & Vision API

In [ ]:
class MinhaStreamListener(tw.StreamListener):

  def on_status(self, status):
    print("Usuário:" , status.user.screen_name)
    print("Texto:" ,status.text)

    # Tem foto
    if 'media' in status.entities:
      # Qual o URL
      url = status.entities['media'][0]['media_url']
      print("URL: ", url)

      # Tem alguma celebridade
      analise_celebridades = client.analyze_image_by_domain("celebrities", url, "en")
      lista_celebridades = [celebridade['name'] for celebridade in analise_celebridades.result["celebrities"]]
      print(lista_celebridades)

      # Descreve a imagem
      descricao = client.describe_image(url,1,"en")
      texto_descricao = descricao.captions[0].text
      print("Descricao: ", texto_descricao)

    print('-----')
    print('\n')

In [ ]:
minhaStream = None # Limpar objeto
minhaStream = tw.Stream(auth = auth, listener=MinhaStreamListener())

In [ ]:
# me seguir
id_usuario = "917548352446791685"
minhaStream.filter(follow=[cliente_id])

## Analisando imagens de famosos

In [ ]:
api.get_user("Mariana23014622").id_str

In [ ]:
#id dos famosos

famosos_id = []

for famoso in famosos:
  famoso_id = api.get_user(famoso).id_str
  famosos_id.append(famoso_id)

In [ ]:
class MinhaStreamListener(tw.StreamListener):

  def on_status(self, status):
    print("Usuário:" , status.user.screen_name)
    print("Texto:" ,status.text)

    if 'media' in status.entities:
      url = status.entities['media'][0]['media_url']
      print("URL: ", url)

      # Celebridades
      analise_celebridades = client.analyze_image_by_domain("celebrities", url, "en")
      lista_celebridades = [celebridade['name'] for celebridade in analise_celebridades.result["celebrities"]]
      print(lista_celebridades)

      # Descricão
      descricao = client.describe_image(url,1,"en")
      texto_descricao = descricao.captions[0].text
      print("Descricao: ", texto_descricao)

      # Quais são as tags
      analize_de_imagem = client.analyze_image(url,VisualFeatureTypes.tags)
      lista_tags = [tag.name for tag in analize_de_imagem.tags]
      print("Tag: ", lista_tags)

      resultados = {
          'usuario': status.user.screen_name,
          'texto': status.text,
          'url': url,
          'celebridades': lista_celebridades,
          'descricao': texto_descricao,
          'tags': lista_tags
      }

      # 'a' de append
      with open('tweets.txt','a') as arquivo:
        # converte dicionario em texto
        arquivo.write(json.dumps(resultados))
        arquivo.write('\n')



    print('-----')
    print('\n')

In [ ]:
minhaStream = None
minhaStream = tw.Stream(auth = auth, listener=MinhaStreamListener())
minhaStream.filter(follow=famosos_id)

# Avaliação dos resultados resultados

In [ ]:
lista_arquivo = []

# r de read
with open("tweets.txt", 'r') as arquivo:
  for linha in arquivo:
    # loads = contrário do dumps
    lista_arquivo.append(json.loads(linha))

lista_arquivo[0]

In [ ]:
imagens = pd.DataFrame(lista_arquivo)
imagens.head()

In [ ]:
pesquisa = 'concert'
# concert está na lista? se sim adiciona nos achados
achados = [(pesquisa in lista) for lista in imagens.tags]

quais são as imagens que tem concert

In [ ]:
imagens[achados]

descrição

In [ ]:
imagens[achados].descricao.iloc[0]

tags

In [ ]:
imagens[achados].tags.iloc[0]

url

In [ ]:
imagens[achados].url.iloc[0]

**Insight**

Fizemos a gravação dos tweets capturados e as análises da Computer Vision API, gravando um arquivo texto localmente.

Uma outra forma de manter as informações seria gravar em um banco de dados. Você pode escolher desde bancos relacionais como MYSQL, PostgreSQL, etc.; ou bancos NOSQL, como MongoDB, Cassandra, etc.

Além disso, como já temos uma conta configurada na Azure para o reconhecimento de imagens, também é possível usar um serviço de bancos de dados gerenciado, ou criar os recursos necessários para usar o seu banco de dados de preferência.